# Challenge 0

In [2]:
import sys
sys.version

'3.6.5 (default, Jun 17 2018, 12:15:43) \n[GCC 4.2.1 Compatible Apple LLVM 8.0.0 (clang-800.0.42.1)]'

In [4]:
! pip freeze
! pip --version

certifi==2018.8.24
pip 18.0 from /Users/ibrahimmuhammad/miniconda3/envs/py35/lib/python3.5/site-packages/pip (python 3.5)


In [1]:
! curl -O https://challenge.blob.core.windows.net/challengefiles/gear_images.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33.4M  100 33.4M    0     0  2516k      0  0:00:13  0:00:13 --:--:-- 4663k


# Challenge 1

In [11]:
import requests

def predict(image_url):
    prediction_url = 'https://southcentralus.api.cognitive.microsoft.com/customvision/v2.0/Prediction/095eac50-bf07-4100-8352-80ff06aee711/url?iterationId=2d6a978e-d4a6-4770-ba4b-9529224b5cc1'
    data = {"Url": image_url}
    headers = {
        'Prediction-Key': 'a99c55c1f6a740c3862bb1ffba0d3d4c',
        'Content-Type': 'application/json'
    }

    response = requests.post(prediction_url, json=data, headers=headers)
    return response.json()

In [13]:
from pprint import pprint
pprint(predict('https://image.sportsmansguide.com/adimgs/l/6/673942i3_ts.jpg'))

{'created': '2018-10-02T17:47:53.5993224Z',
 'id': 'd5eeb4ef-3a34-41dc-bb5c-bc775a518fd0',
 'iteration': '2d6a978e-d4a6-4770-ba4b-9529224b5cc1',
 'predictions': [{'probability': 1.0,
                  'tagId': 'a91fc173-47bc-4d62-b395-4295884004a3',
                  'tagName': 'insulated_jacket'},
                 {'probability': 7.454575e-21,
                  'tagId': 'f6010827-8174-4eb6-8523-f1d4b913318e',
                  'tagName': 'hardshell_jacket'}],
 'project': '095eac50-bf07-4100-8352-80ff06aee711'}
